# `virtual`: How to virtualize Redset

In [ ]:
%load_ext autoreload
%autoreload 2

import virtual

parquet_filepath = 's3://redshift-downloads/redset/serverless/full.parquet'

### One-shot table virtualization

In [ ]:
# Virtualize Parquet file stored on S3.
virtual.to_format(parquet_filepath, 'redset_virtual.parquet', prefix='demo-debug/')

### Compare to vanilla `Parquet`

In [ ]:
import os
import demo_util
%pip install requests

table = os.path.splitext(os.path.basename(parquet_filepath))[0]
print(f'[{table}] Parquet size: {demo_util.get_file_size(parquet_filepath) / 1_000_000} MB')
print(f'[{table}] Virtual size: {os.path.getsize('redset_virtual.parquet') / 1_000_000} MB')

### Query the vanilla Parquet file

In [ ]:
import duckdb

duckdb.sql(f'select num_scans, count(*) from read_parquet("{parquet_filepath}") group by num_scans order by num_scans;').fetchdf()

### Query the virtualized Parquet file

In [ ]:
virtual.query(
  'select num_scans, count(*) from read_parquet("redset_virtual.parquet") group by num_scans order by num_scans',
  engine = 'duckdb'
)